<a href="https://colab.research.google.com/github/sarvesh23110076/CS203_Assign_7/blob/main/CS203_ASSIGN7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<B>CS203 : Software Tools and Techniques fo AI </B><br>
<B>LAB7 Assignment</B><br>
<B>Group Number : 08<Br>
<B>Group Members :</B><br>
<B>Sarvesh Pravin Chaudhari (<I>23110076</I>)<br>
Afraz Azeem (<I>23110019</I>)<B>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter

**1. Dataset Preparation (10%)**

- Load the training dataset and test data (Dataset 1)<br>
Use 20% of the training dataset as the validation set.

In [ ]:
# Loading Dataset 1
url1="https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/train.tsv"
url2="https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/test.tsv"
# url3="https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/dev.tsv"

data1=pd.read_csv(url1, sep='\t', header=None, names=['sentence', 'label'])
data1_train, data1_val=train_test_split(data1, test_size=0.2, random_state=42)
data1_test=pd.read_csv(url2, sep='\t', header=None, names=['sentence', 'label'])
# data1_val=pd.read_csv(url3, sep='\t', header=None, names=['sentence', 'label'])

print(f"Training samples: {len(data1_train)}")
print(f"Test samples: {len(data1_test)}")
print(f"Validation samples: {len(data1_val)}")

Training samples: 5536
Test samples: 1821
Validation samples: 1384


- Also, the IMDB dataset (Dataset 2) can be used for continual learning.<br>
Use 20% of the training dataset as the validation set.

In [ ]:
# Loading Dataset 2
url4="https://raw.githubusercontent.com/Ankit152/IMDB-sentiment-analysis/master/IMDB-Dataset.csv"
data2=pd.read_csv(url4)

data2_temp, data2_test=train_test_split(data2, test_size=0.2, random_state=42)
data2_train, data2_val = train_test_split(data2_temp, test_size=0.2, random_state=42)

print("\nDataset 2:")
print(f"Training samples: {len(data2_train)}")
print(f"Test samples: {len(data2_test)}")
print(f"Validation samples: {len(data2_val)}")


Dataset 2:
Training samples: 32000
Test samples: 10000
Validation samples: 8000


**2. Construct a Multi-Layer Perceptron (MLP) model. (20%)**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer

- The parameter should be with:<br>
hidden_sizes=[512, 256, 128, 64]<br>
Output should have two labels.

In [ ]:
# Defining the MLP model
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_sizes=[512, 256, 128, 64], output_size=2, dropout_prob=0.3):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.fc4 = nn.Linear(hidden_sizes[2], hidden_sizes[3])
        self.fc5 = nn.Linear(hidden_sizes[3], output_size)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        return self.fc5(x)

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

- Count the number of trainable parameters in the model using the automated function.

In [ ]:
model=MLPModel(10000)
print(f"Number of trainable parameters in the model: {model.count_parameters()}")

Number of trainable parameters in the model: 5293122


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

3. Implement case 1: Bag-of-words (20%)<br>
- Implement the bag-of-words (max_features=10000).<br>
- Hint: from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import DataLoader, TensorDataset

# Bag-of-Words Vectorization
vectorizer = CountVectorizer(max_features=10000)

# Transforming sentences into BoW features
X_train_bow = vectorizer.fit_transform(data1_train['sentence']).toarray()
X_val_bow = vectorizer.transform(data1_val['sentence']).toarray()
X_test_bow = vectorizer.transform(data1_test['sentence']).toarray()

# Converting labels to tensors
y_train_bow = torch.tensor(data1_train['label'].values)
y_val_bow = torch.tensor(data1_val['label'].values)
y_test_bow = torch.tensor(data1_test['label'].values)

# Creating DataLoaders for BoW
train_dataset_bow = TensorDataset(torch.tensor(X_train_bow, dtype=torch.float32), y_train_bow)
val_dataset_bow = TensorDataset(torch.tensor(X_val_bow, dtype=torch.float32), y_val_bow)
test_dataset_bow = TensorDataset(torch.tensor(X_test_bow, dtype=torch.float32), y_test_bow)

train_loader_bow = DataLoader(train_dataset_bow, batch_size=32, shuffle=True)
val_loader_bow = DataLoader(val_dataset_bow, batch_size=32)
test_loader_bow = DataLoader(test_dataset_bow, batch_size=32)

print("Bag-of-Words features prepared.")

Bag-of-Words features prepared.


<B>4. Implement case 2: Construct a function to use embeddings on the same model. (20%)<br></B>
- Use the model: meta-llama/Llama-3.1-8B or use bert-base-uncased if facing issues with the GPU constraints.
- TIPS:<br>
You can use the distilled version, gather embeddings for 200 samples, and even reduce the precision to deal with computing issues!
- Hints:<br>
self.tokenizer = AutoTokenizer.from_pretrained(model_name)
     self.model = AutoModel.from_pretrained(model_name).to(device)
     self.embedding_size = self.model.config.hidden_size
     self.model_loaded = True

In [ ]:
from transformers import AutoTokenizer, AutoModel

class EmbeddingExtractor:
    def __init__(self, model_name='bert-base-uncased', device=None):
        self.device = device if device else ('cuda' if torch.cuda.is_available() else 'cpu')

        # Initializing tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)

    def get_embeddings(self, sentences):
        inputs = self.tokenizer(sentences, padding=True, truncation=True, return_tensors="pt", max_length=512).to(self.device)

        # Extracting embeddings using [CLS] token representation
        with torch.no_grad():
            outputs = self.model(**inputs)

        return outputs.last_hidden_state[:, 0, :].cpu().numpy()

embedding_extractor = EmbeddingExtractor()

# Extracting embeddings for train/validation/test datasets
X_train_emb = embedding_extractor.get_embeddings(data1_train['sentence'].tolist()[:200])  # First 200 samples due to GPU constraints
X_val_emb = embedding_extractor.get_embeddings(data1_val['sentence'].tolist()[:200])
X_test_emb = embedding_extractor.get_embeddings(data1_test['sentence'].tolist()[:200])

# Convert labels to tensors
y_train_emb = torch.tensor(data1_train['label'].values[:200])
y_val_emb = torch.tensor(data1_val['label'].values[:200])
y_test_emb = torch.tensor(data1_test['label'].values[:200])

# Create DataLoaders for embeddings
train_dataset_emb = TensorDataset(torch.tensor(X_train_emb, dtype=torch.float32), y_train_emb)
val_dataset_emb = TensorDataset(torch.tensor(X_val_emb, dtype=torch.float32), y_val_emb)
test_dataset_emb = TensorDataset(torch.tensor(X_test_emb, dtype=torch.float32), y_test_emb)

train_loader_emb = DataLoader(train_dataset_emb, batch_size=32, shuffle=True)
val_loader_emb = DataLoader(val_dataset_emb, batch_size=32)
test_loader_emb = DataLoader(test_dataset_emb, batch_size=32)

print("Embedding-based features prepared")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Embedding-based features prepared


<B>5. Train the model with 10 epochs and create the best-performing model (checkpoint.pt) on the Dataset 1. (10%)<br></B>
- Get the validation accuracy.

In [ ]:
import gzip
import json

#Function to save the best model at a checkpoint in compressed form
def save_checkpoint(state,filename,compress=True):
    if compress:
        with gzip.open(filename + '.gz', 'wb') as f:
            torch.save(state, f)
    else:
        torch.save(state, filename)

#Function to load saved best model from a checkpoint
def load_checkpoint(filename, compress=True):
    if compress:
        with gzip.open(filename + '.gz', 'rb') as f:
            state = torch.load(f)
    else:
        state = torch.load(filename)
    return state

In [ ]:
#Function to plot loss curves and save to tensorboard
import matplotlib.pyplot as plt

def plot_loss_curves(train_losses, val_losses,title):
    epochs = range(1, len(train_losses) + 1)

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    axes[0].plot(epochs, train_losses, marker='o', linestyle="--", color='b')
    axes[0].set_title(f"{title} - Train Loss Curve")
    axes[0].set_xlabel("Epochs")
    axes[0].set_ylabel("Train Loss")
    axes[0].legend()
    axes[0].grid(True)

    axes[1].plot(epochs, val_losses, marker='s', linestyle="-", color='m')
    axes[1].set_title(f"{title} - Valid Loss Curve")
    axes[1].set_xlabel("Epochs")
    axes[1].set_ylabel("Validation Loss")
    axes[1].legend()
    axes[1].grid(True)

    plt.tight_layout()
    return fig

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer,epochs,save_path,model_title,dataset,resume=False):
    writer=SummaryWriter(f'runs/{model_title}/{dataset}')

    dummy_input = torch.randn(1, model.fc1.in_features).to(device)  # Dummy input for logging the graph
    writer.add_graph(model, dummy_input)


    best_val_acc=0.0
    if resume:
        checkpoint = load_checkpoint(save_path)
        model.load_state_dict(checkpoint['model_state_dict'])

    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()
        train_loss = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        writer.add_scalar('Loss/train', avg_train_loss, epoch)

        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_acc=correct/len(val_loader.dataset)
        val_losses.append(avg_val_loss)

        writer.add_scalar('Loss/val', avg_val_loss, epoch)
        writer.add_scalar('Accuracy/val', val_acc, epoch)

        hyperparams = {
            'input_size': model.fc1.in_features,
            'hidden_sizes': str([model.fc1.out_features, model.fc2.out_features, model.fc3.out_features, model.fc4.out_features]),  # Convert list to string
            'output_size': model.fc5.out_features,
            'dropout_prob': float(model.dropout.p),
            'learning_rate': float(optimizer.param_groups[0]['lr']),
            'epochs': int(epochs)
        }

        writer.add_hparams(
            hparam_dict=hyperparams,
            metric_dict={
                'Loss/val': avg_val_loss,
                'Accuracy/val': val_acc
            }
        )

        if val_acc > best_val_acc:
            best_val_acc=val_acc
            checkpoint = {
                'model_state_dict': model.state_dict(),
            }
            save_checkpoint(checkpoint, save_path)

    writer.add_figure('Loss Curves', plot_loss_curves(train_losses, val_losses,model_title), global_step=0)
    writer.close()
    print(hyperparams)

In [ ]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [ ]:
# Defining device globally
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiating MLP model for BoW features (input size is 10000 for BoW)
model_bow = MLPModel(input_size=10000).to(device)

criterion_bow = nn.CrossEntropyLoss()
optimizer_bow = torch.optim.Adam(model_bow.parameters(), lr=0.001)

#training the model and saving the best model at checkpoint
train_model(model_bow, train_loader_bow, val_loader_bow, criterion_bow,optimizer_bow,10,'checkpoint_bow.pt','Bag-of-Words','Dataset1')
test_model(model_bow, test_loader_bow)

{'input_size': 10000, 'hidden_sizes': '[512, 256, 128, 64]', 'output_size': 2, 'dropout_prob': 0.3, 'learning_rate': 0.001, 'epochs': 10}
Test Accuracy: 78.80%


In [ ]:
# Instantiating MLP model for embeddings (input size is 768 for BERT embeddings)
model_emb = MLPModel(input_size=768).to(device)

criterion_emb = nn.CrossEntropyLoss()
optimizer_emb = torch.optim.Adam(model_emb.parameters(), lr=0.001)

#training the model and saving the best model at checkpoint
train_model(model_emb, train_loader_emb, val_loader_emb, criterion_emb, optimizer_emb,10,'checkpoint_emb.pt','Embeddings','Dataset1')
test_model(model_emb, test_loader_emb)

{'input_size': 768, 'hidden_sizes': '[512, 256, 128, 64]', 'output_size': 2, 'dropout_prob': 0.3, 'learning_rate': 0.001, 'epochs': 10}
Test Accuracy: 75.50%


<B>6. Use the checkpoint from before and train on the IMDB dataset (Dataset 2). (10%)<br></B>
- Use the following parameters:<br>
criterion = nn.CrossEntropyLoss()<br>
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Smaller learning rate

In [ ]:
from sklearn.preprocessing import LabelEncoder

#Function to get encoding for labels
def encode_labels(series):
    label_encoder=LabelEncoder()
    encoded_labels=label_encoder.fit_transform(series.astype(str))
    return torch.tensor(encoded_labels,dtype=torch.long)

Preparing IMDB dataset to train of Bag-of-Words Model

In [ ]:
vectorizer = CountVectorizer(max_features=10000)

X_train_bow = vectorizer.fit_transform(data2_train['review']).toarray()
X_val_bow = vectorizer.transform(data2_val['review']).toarray()
X_test_bow = vectorizer.transform(data2_test['review']).toarray()

# Converting labels to tensors
y_train_bow = encode_labels(data2_train['sentiment'].values)
y_val_bow = encode_labels(data2_val['sentiment'].values)
y_test_bow = encode_labels(data2_test['sentiment'].values)

# Creating DataLoaders for BoW
train_dataset_bow = TensorDataset(torch.tensor(X_train_bow, dtype=torch.float32), y_train_bow)
val_dataset_bow = TensorDataset(torch.tensor(X_val_bow, dtype=torch.float32), y_val_bow)
test_dataset_bow = TensorDataset(torch.tensor(X_test_bow, dtype=torch.float32), y_test_bow)

train_loader_bow = DataLoader(train_dataset_bow, batch_size=32, shuffle=True)
val_loader_bow = DataLoader(val_dataset_bow, batch_size=32)
test_loader_bow = DataLoader(test_dataset_bow, batch_size=32)

Preparing IMDB dataset to train of Embeddings Model

In [ ]:
embedding_extractor = EmbeddingExtractor()

# Extracting embeddings for train/validation/test datasets
X_train_emb = embedding_extractor.get_embeddings(data2_train['review'].tolist()[:200])  # First 200 samples due to GPU constraints
X_val_emb = embedding_extractor.get_embeddings(data2_val['review'].tolist()[:200])
X_test_emb = embedding_extractor.get_embeddings(data2_test['review'].tolist()[:200])

# Convert labels to tensors
y_train_emb = encode_labels(data2_train['sentiment'].values[:200])
y_val_emb = encode_labels(data2_val['sentiment'].values[:200])
y_test_emb = encode_labels(data2_test['sentiment'].values[:200])

# Create DataLoaders for embeddings
train_dataset_emb = TensorDataset(torch.tensor(X_train_emb, dtype=torch.float32), y_train_emb)
val_dataset_emb = TensorDataset(torch.tensor(X_val_emb, dtype=torch.float32), y_val_emb)
test_dataset_emb = TensorDataset(torch.tensor(X_test_emb, dtype=torch.float32), y_test_emb)

train_loader_emb = DataLoader(train_dataset_emb, batch_size=32, shuffle=True)
val_loader_emb = DataLoader(val_dataset_emb, batch_size=32)
test_loader_emb = DataLoader(test_dataset_emb, batch_size=32)

8. Compute the validation loss and accuracy on the validation set of the Dataset 1 and IMDB dataset. (10%)

Training Bag-of-Words Model

In [ ]:
# Defining device globally
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiating MLP model for BoW features (input size is 10000 for BoW)
model_bow = MLPModel(input_size=10000).to(device)

criterion_bow = nn.CrossEntropyLoss()
optimizer_bow = torch.optim.Adam(model_bow.parameters(), lr=0.0001)

train_model(model_bow, train_loader_bow, val_loader_bow, criterion_bow, optimizer_bow,10,'checkpoint_bow.pt','Bag-of-Words','IMDB_dataset',True)
test_model(model_bow, test_loader_bow)

<ipython-input-10-13a70b0b6b2e>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f)


{'input_size': 10000, 'hidden_sizes': '[512, 256, 128, 64]', 'output_size': 2, 'dropout_prob': 0.3, 'learning_rate': 0.0001, 'epochs': 10}
Test Accuracy: 87.93%


Training Embeddings Model

In [ ]:
# Instantiating MLP model for embeddings (input size is 768 for BERT embeddings)
model_emb = MLPModel(input_size=X_train_emb.shape[1]).to(device)

criterion_emb = nn.CrossEntropyLoss()
optimizer_emb = torch.optim.Adam(model_emb.parameters(), lr=0.0001)

train_model(model_emb, train_loader_emb, val_loader_emb, criterion_emb, optimizer_emb,10,'checkpoint_emb.pt','Embeddings','IMDB_dataset',True)
test_model(model_emb, test_loader_emb)

<ipython-input-10-13a70b0b6b2e>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f)


{'input_size': 768, 'hidden_sizes': '[512, 256, 128, 64]', 'output_size': 2, 'dropout_prob': 0.3, 'learning_rate': 0.0001, 'epochs': 10}
Test Accuracy: 76.50%


9. Submission Requirements<br>
- Python code for training, testing, and evaluation.
- Screenshots of the following displaying:<br>
Model architecture.<br>
Hyperparameters.<br>
Logged metrics.<br>
Final evaluation results.<br>
Confusion matrix visualization.<br>
Training and validation loss curves.<br>

In [ ]:
checkpoint = load_checkpoint("checkpoint_bow.pt")
model_bow.load_state_dict(checkpoint['model_state_dict'])

checkpoint = load_checkpoint("checkpoint_emb.pt")
model_emb.load_state_dict(checkpoint['model_state_dict'])

<ipython-input-10-13a70b0b6b2e>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f)


<All keys matched successfully>

Final evaluation results.

In [ ]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import pandas as pd

def evaluation_metrics(model,loader,model_title):
    writer=SummaryWriter(f'runs/{model_title}')
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute evaluation metrics
    accuracy = accuracy_score(y_true, y_pred) * 100
    precision = precision_score(y_true, y_pred, average="weighted")  # Weighted for class imbalance
    recall = recall_score(y_true, y_pred, average="weighted")
    f1 = f1_score(y_true, y_pred, average="weighted")

    writer.add_scalar('Final Evaluation/Accuracy', accuracy, 0)
    writer.add_scalar('Final Evaluation/Precision', precision, 0)
    writer.add_scalar('Final Evaluation/Recall', recall, 0)
    writer.add_scalar('Final Evaluation/F1-Score', f1, 0)

    writer.close()

    result={
        "Metrics":['Accuracy','Precision','Recall','F1-Score'],
        "Values":[accuracy,precision,recall,f1]
    }
    return pd.DataFrame(result).set_index("Metrics")

In [ ]:
results=evaluation_metrics(model_bow,test_loader_bow,'Bag-of-Words')
results

,Values
Metrics,
Accuracy,89.320000
Precision,0.893463
Recall,0.893200
F1-Score,0.893194


In [ ]:
results=evaluation_metrics(model_emb,test_loader_emb,'Embeddings')
results

,Values
Metrics,
Accuracy,76.000000
Precision,0.765210
Recall,0.760000
F1-Score,0.759856


Confusion matrix visualization.

In [ ]:
def get_class_names_from_loader(loader):
    dataset = loader.dataset

    if hasattr(dataset, 'classes'):
        return dataset.classes
    elif hasattr(dataset, 'class_to_idx'):
        return list(dataset.class_to_idx.keys())

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model, loader, class_labels, model_name):
    writer=SummaryWriter(f'runs/{model_name}')
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Display confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_labels)

    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(cmap="Blues", values_format="d",ax=ax)
    plt.title(f"Confusion Matrix - {model_name}_Model")
    plt.xticks(rotation=45)
    plt.tight_layout()
    writer.add_figure(f'Confusion Matrix',fig, global_step=0)  # Save figure
    plt.close(fig)
    writer.close()

In [ ]:
plot_confusion_matrix(model_bow,test_loader_bow,get_class_names_from_loader(test_loader_bow),"Bag-of-Words")

In [ ]:
plot_confusion_matrix(model_emb,test_loader_emb,get_class_names_from_loader(test_loader_emb),"Embeddings")

In [ ]:
import shutil

output_dir = "/kaggle/working/runs"
zip_path = "/kaggle/working/runs.zip"

shutil.make_archive(zip_path.replace(".zip", ""), 'zip', output_dir)

'/kaggle/working/runs.zip'